In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [18]:
data_url = "https://www.hoopshype.com/salaries/players/"
driver = webdriver.Firefox()
driver.get(data_url)
html = driver.page_source
soup = BeautifulSoup(html)
for tag in soup.find_all('tr'):
    print(tag.prettify())

<tr>
 <th colspan="1">
  <span>
  </span>
 </th>
 <th colspan="1">
  <span>
   Player
  </span>
 </th>
 <th class="RLrCiX__RLrCiX" colspan="1">
  <span>
   2025-26
  </span>
 </th>
 <th class="RLrCiX__RLrCiX" colspan="1">
  <span>
   2026-27
  </span>
 </th>
 <th class="RLrCiX__RLrCiX" colspan="1">
  <span>
   2027-28
  </span>
 </th>
 <th class="RLrCiX__RLrCiX" colspan="1">
  <span>
   2028-29
  </span>
 </th>
</tr>

<tr>
 <td class="">
  1
 </td>
 <td class="">
  <div class="SnFxMh__SnFxMh">
   <div class="XyAr3f__XyAr3f c62zCr__c62zCr">
    <img alt="" class="" decoding="auto" loading="eager" src="https://www.gannett-cdn.com/content-pipeline-sports-images/sports2/nba/logos/9.png?format=png8&amp;auto=webp&amp;quality=85,75&amp;width=30" srcset="https://www.gannett-cdn.com/content-pipeline-sports-images/sports2/nba/logos/9.png?format=png8&amp;auto=webp&amp;quality=85,75&amp;width=60 2x" width="30"/>
   </div>
   <a class="_9hWOUd__9hWOUd" href="/salaries/players/stephen-curry/338365/"

In [158]:
data_url = "https://www.hoopshype.com/salaries/players/"
driver = webdriver.Firefox()
driver.get(data_url)
html = driver.page_source
soup = BeautifulSoup(html)

In [4]:
year_buttons = driver.find_elements(By.CLASS_NAME,"ULq-fz__ULq-fz")
year_buttons[0].click()
all_years = driver.find_elements(By.CLASS_NAME,"_2qaD9L__2qaD9L")
# for yr in all_years:
#     yr.click()
#     time.sleep(3)

In [6]:
data_urls = [yr.get_attribute("href") for yr in all_years]
wait = WebDriverWait(driver, 10) 
data_urls

['https://www.hoopshype.com/salaries/players//?season=2025',
 'https://www.hoopshype.com/salaries/players//?season=2024',
 'https://www.hoopshype.com/salaries/players//?season=2023',
 'https://www.hoopshype.com/salaries/players//?season=2022',
 'https://www.hoopshype.com/salaries/players//?season=2021',
 'https://www.hoopshype.com/salaries/players//?season=2020',
 'https://www.hoopshype.com/salaries/players//?season=2019',
 'https://www.hoopshype.com/salaries/players//?season=2018',
 'https://www.hoopshype.com/salaries/players//?season=2017',
 'https://www.hoopshype.com/salaries/players//?season=2016',
 'https://www.hoopshype.com/salaries/players//?season=2015',
 'https://www.hoopshype.com/salaries/players//?season=2014',
 'https://www.hoopshype.com/salaries/players//?season=2013',
 'https://www.hoopshype.com/salaries/players//?season=2012',
 'https://www.hoopshype.com/salaries/players//?season=2011',
 'https://www.hoopshype.com/salaries/players//?season=2010',
 'https://www.hoopshype.

In [ ]:
salary_df = pd.DataFrame()
row_idx = 0

for data_url in data_urls[:8]:
    driver = webdriver.Firefox()
    driver.get(data_url)
    html = driver.page_source
    soup = BeautifulSoup(html)
    table_pg_1 = soup.find_all('tr')

    headers_raw = table_pg_1[0].find_all(class_="RLrCiX__RLrCiX")
    headers = [header.text for header in headers_raw]
    page_num_count = 3
    while page_num_count != 2:
        retries = 0
        page_success = False

        while retries < MAX_RETRIES and not page_success:
            try:
                html = driver.page_source
                soup = BeautifulSoup(html)
                table = soup.find_all('tr')
                page_num = set(driver.find_elements(By.CLASS_NAME,"l98ZPp__l98ZPp")[0].text.split())
                page_num_count = len(page_num)
                print(page_num)

                for player in table[1:]:
                    try:
                        player_salaries = player.find_all(class_="RLrCiX__RLrCiX")
                        name = player.find(class_="_0cD6l-__0cD6l-").text
                    except Exception as e:
                        print(f"Error parsing player row: {e}")
                        continue
                        
                    salary_df.loc[row_idx, "name"] = name
                    for i in range(len(player_salaries)):
                        salary_df.loc[row_idx, headers[i]] = player_salaries[i].text
                    row_idx += 1

                page_success = True

            except Exception as e:
                retries += 1
                print(f"Error processing page (attempt {retries}/{MAX_RETRIES}): {e}")
                time.sleep(1)
                if retries >= MAX_RETRIES:
                    print("Max retries reached for this page, skipping.")

        print(salary_df.shape)
        print(salary_df.iloc[-5:])
        
        buttons = driver.find_elements(By.CLASS_NAME, "hd3Vfp__hd3Vfp")
        buttons[1].click()


        time.sleep(2)

    driver.quit()
    salary_df.to_csv("data/00-raw/salary_data_raw.csv", index=False)
    time.sleep(10)

{'1', '31', 'of'}
(20, 5)
                name      2025-26       2026-27       2027-28      2028-29
15        Trae Young  $46,394,100  P$48,967,380             -            -
16   Lauri Markkanen  $46,394,100   $46,113,154   $49,824,681  $53,536,209
17  Donovan Mitchell  $46,394,100   $50,105,628  P$53,817,156            -
18   Cade Cunningham  $46,394,100   $50,105,628   $53,817,156  $57,528,684
19       Evan Mobley  $46,394,100   $50,105,628   $53,817,156  $57,528,684
{'31', '2', 'of'}
(40, 5)
              name      2025-26      2026-27       2027-28       2028-29
35  Brandon Ingram  $38,095,238  $40,000,000  P$41,904,762             -
36     LaMelo Ball  $37,958,760  $40,770,520   $43,582,280   $46,394,040
37    Tyrese Maxey  $37,958,760  $40,770,520   $43,582,280   $46,394,040
38    De'Aaron Fox  $37,096,620  $51,033,600   $55,116,288   $59,198,976
39     Bam Adebayo  $37,096,620  $51,935,268   $56,090,089  P$60,244,910
{'3', '31', 'of'}
(60, 5)
               name      2025-26  